# YOLOv11 Model Evaluation for Single-Class Vehicle Detection

**Research Project**: Comparative Analysis of CNN vs Vision Transformer

**Author**: Abdullah Waraich


## 1. Environment Setup

First, let's check GPU availability and install the required libraries.

In [ ]:
# Check GPU availability
!nvidia-smi


In [ ]:
# Install required libraries
!pip install -q ultralytics supervision scikit-learn seaborn

print("Libraries installed successfully!")


## 2. Upload Model and Dataset

Upload the trained YOLOv11 model (.pt file) and test dataset.

In [ ]:
from google.colab import files
import os
import zipfile

print("Upload yolo model (.pt file) and dataset zip file:")

uploaded = files.upload()

# Process uploaded files
model_path = None
dataset_path = None

for filename in uploaded.keys():
    if filename.endswith('.pt'):
        model_path = filename
        print(f"Model found: {filename}")
    elif filename.endswith('.zip'):
        # Extract dataset
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('test_dataset')
        dataset_path = 'test_dataset'
        print(f"Dataset extracted to: {dataset_path}")

        # Show dataset structure
        print("\n Dataset structure:")
        for root, dirs, files in os.walk(dataset_path):
            level = root.replace(dataset_path, '').count(os.sep)
            indent = ' ' * 2 * level
            print(f"{indent}{os.path.basename(root)}/")
            subindent = ' ' * 2 * (level + 1)
            for file in files[:3]:  # Show first 3 files
                print(f"{subindent}{file}")
            if len(files) > 3:
                print(f"{subindent}... and {len(files)-3} more files")

if not model_path:
    print(" No .pt model file found. Please upload your trained YOLOv11 model.")
if not dataset_path:
    print(" No dataset zip file found. Please upload your test dataset.")

## 3. Load Model and Analyze Dataset

Now let's load the YOLOv11 model and analyze the test dataset structure for single-class vehicle detection.

In [ ]:
import torch
from ultralytics import YOLO
import supervision as sv
from PIL import Image
import numpy as np
import glob
from pathlib import Path

# Load YOLOv11 model
model = YOLO(model_path)

# Get model info
print(f"Model loaded successfully!")
print(f"Model classes: {model.names}")
print(f"Number of classes: {len(model.names)}")

# Define single class for evaluation
CLASS_NAMES = ['vehicle']  # Single class
print(f"Target class for evaluation: {CLASS_NAMES[0]}")

# Analyze test dataset
images_path = os.path.join(dataset_path, 'images')
labels_path = os.path.join(dataset_path, 'labels')

# Get list of test images
image_files = glob.glob(os.path.join(images_path, '*'))
image_files = [f for f in image_files if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

print(f"\n Found {len(image_files)} total test images")
print(f" Images directory: {images_path}")
print(f" Labels directory: {labels_path}")


if len(image_files) == 0:
    print("No image files found.")
else:
    print("Dataset loaded successfully!")

## 4. Run Inference with Timing Analysis

Let's run inference on all test images and measure the time taken for single-class vehicle detection.

In [ ]:
import time
from tqdm import tqdm

print(f"Processing {len(image_files)} images")

# Storage for results
all_predictions = []
all_ground_truth = []
inference_times = []
image_paths = []

# Process each image
for img_path in tqdm(image_files, desc="Running inference"):
    try:
        # Load image
        image = Image.open(img_path)
        image_paths.append(img_path)

        # Measure inference time
        start_time = time.time()
        results = model(img_path, verbose=False)
        end_time = time.time()

        inference_time = end_time - start_time
        inference_times.append(inference_time)

        # Extract predictions
        detections = results[0].boxes

        if detections is not None and len(detections) > 0:
            # Convert to supervision format
            xyxy = detections.xyxy.cpu().numpy()
            confidence = detections.conf.cpu().numpy()
            # For single class, all detections are class 0 (vehicle)
            class_id = np.zeros(len(xyxy), dtype=int)

            pred_detection = sv.Detections(
                xyxy=xyxy,
                confidence=confidence,
                class_id=class_id
            )
        else:
            # No detections
            pred_detection = sv.Detections.empty()

        all_predictions.append(pred_detection)

        # Load ground truth annotations
        img_name = Path(img_path).stem
        label_path = os.path.join(labels_path, f"{img_name}.txt")

        if os.path.exists(label_path):
            # Getting YOLO format labels
            gt_boxes = []
            gt_classes = []

            with open(label_path, 'r') as f:
                lines = f.readlines()

            img_width, img_height = image.size

            for line in lines:
                if line.strip():
                    parts = line.strip().split()
                    # For single class, class_id is always 0 (vehicle)
                    class_id = 0
                    x_center = float(parts[1]) * img_width
                    y_center = float(parts[2]) * img_height
                    width = float(parts[3]) * img_width
                    height = float(parts[4]) * img_height

                    # Convert to xyxy format
                    x1 = x_center - width / 2
                    y1 = y_center - height / 2
                    x2 = x_center + width / 2
                    y2 = y_center + height / 2

                    gt_boxes.append([x1, y1, x2, y2])
                    gt_classes.append(class_id)

            if gt_boxes:
                gt_detection = sv.Detections(
                    xyxy=np.array(gt_boxes),
                    class_id=np.array(gt_classes)
                )
            else:
                gt_detection = sv.Detections.empty()
        else:
            gt_detection = sv.Detections.empty()

        all_ground_truth.append(gt_detection)

    except Exception as e:
        print(f"Error processing {img_path}: {e}")
        continue

print(f"\n Inference completed")
print(f" Average inference time: {np.mean(inference_times):.4f} seconds")
print(f" Average FPS: {1/np.mean(inference_times):.2f}")
print(f" Processed {len(all_predictions)} images successfully")


## 5. Calculate Performance Metrics

Calculate comprehensive performance metrics for single-class vehicle detection.

In [ ]:
from supervision.metrics import MeanAveragePrecision
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix
import pandas as pd

print("Calculating Performance Metrics")

# Calculate mAP using supervision
map_metric = MeanAveragePrecision()
map_result = map_metric.update(all_predictions, all_ground_truth).compute()

print("MEAN AVERAGE PRECISION (mAP) RESULTS:")
print(f"mAP@0.5: {map_result.map50:.3f}")
print(f"mAP@0.5:0.95: {map_result.map50_95:.3f}")
print(f"mAP@0.75: {map_result.map75:.3f}")

# Calculate precision, recall, F1 Score for "vehicle" class
print("\n PRECISION, RECALL, F1-SCORE ANALYSIS:")

# Create binary classification arrays for vehicle detection
all_pred_binary = []
all_true_binary = []

for pred, gt in zip(all_predictions, all_ground_truth):
    # Binary: has vehicle or not
    has_prediction = len(pred) > 0
    has_ground_truth = len(gt) > 0

    all_pred_binary.append(1 if has_prediction else 0)
    all_true_binary.append(1 if has_ground_truth else 0)

# Convert to numpy arrays
all_pred_binary = np.array(all_pred_binary)
all_true_binary = np.array(all_true_binary)

# Calculate binary metrics
if np.sum(all_true_binary) > 0:  # Only if we have positive samples
    precision, recall, f1, _ = precision_recall_fscore_support(
        all_true_binary, all_pred_binary, average='binary', zero_division=0
    )

    print(f"\n{CLASS_NAMES[0].upper()} DETECTION:")
    print(f"  Precision: {precision:.3f}")
    print(f"  Recall: {recall:.3f}")
    print(f"  F1-Score: {f1:.3f}")

    # Calculate additional metrics
    total_images = len(all_true_binary)
    true_positives = np.sum((all_true_binary == 1) & (all_pred_binary == 1))
    false_positives = np.sum((all_true_binary == 0) & (all_pred_binary == 1))
    false_negatives = np.sum((all_true_binary == 1) & (all_pred_binary == 0))
    true_negatives = np.sum((all_true_binary == 0) & (all_pred_binary == 0))

    accuracy = (true_positives + true_negatives) / total_images

    print(f"\n DETAILED METRICS:")
    print(f"  Accuracy: {accuracy:.3f}")
    print(f"  True Positives: {true_positives}")
    print(f"  False Positives: {false_positives}")
    print(f"  False Negatives: {false_negatives}")
    print(f"  True Negatives: {true_negatives}")

else:
    print(f"\n{CLASS_NAMES[0].upper()}: No ground truth instances found")
    precision = recall = f1 = accuracy = 0.0



## 6. Generate Confusion Matrix

Create and visualize the confusion matrix for vehicle detection.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

# Create confusion matrix for binary classification
print("Generating Confusion Matrix for Vehicle Detection")

# Create binary classes
binary_classes = ['No Vehicle', 'Vehicle']

# Generate confusion matrix
cm = confusion_matrix(all_true_binary, all_pred_binary, labels=[0, 1])

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=binary_classes,
            yticklabels=binary_classes)
plt.title('YOLOv11 Confusion Matrix\nSatellite Vehicle Detection', fontsize=14, fontweight='bold')
plt.xlabel('Predicted', fontsize=12)
plt.ylabel('True', fontsize=12)
plt.tight_layout()
plt.show()

# Print classification report
print("\n DETAILED CLASSIFICATION REPORT:")
print(classification_report(all_true_binary, all_pred_binary,
                          target_names=binary_classes,
                          zero_division=0))

print(f"\n Summary of Statistics:")
print(f"  Total images: {len(all_true_binary)}")
print(f"  Images with vehicles (GT): {np.sum(all_true_binary)}")
print(f"  Images with vehicles (Pred): {np.sum(all_pred_binary)}")
print(f"  Detection accuracy: {accuracy:.3f}")

## 7. Inference Time Analysis

Analyze the computational efficiency of the YOLOv11 model.

In [ ]:
print("YOLOV11 INFERENCE TIME ANALYSIS")

# Calculate timing statistics
avg_time = np.mean(inference_times)
std_time = np.std(inference_times)
min_time = np.min(inference_times)
max_time = np.max(inference_times)
fps = 1 / avg_time

print(f"Timing Statistics:")
print(f"  Average time per image: {avg_time:.4f} seconds")
print(f"  Standard deviation: {std_time:.4f} seconds")
print(f"  Minimum time: {min_time:.4f} seconds")
print(f"  Maximum time: {max_time:.4f} seconds")
print(f"  Average FPS: {fps:.2f}")

# Plot inference time distribution
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.hist(inference_times, bins=30, alpha=0.7, color='skyblue', edgecolor='black')
plt.axvline(avg_time, color='red', linestyle='--',
           label=f'Mean: {avg_time:.4f}s')
plt.xlabel('Inference Time (seconds)')
plt.ylabel('Frequency')
plt.title('YOLOv11 Inference Time Distribution')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(inference_times[:100], marker='o', markersize=2, alpha=0.7)
plt.axhline(avg_time, color='red', linestyle='--',
           label=f'Mean: {avg_time:.4f}s')
plt.xlabel('Image Index')
plt.ylabel('Inference Time (seconds)')
plt.title('Inference Time per Image (First 100)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 8. Visual Results Analysis

Visualize detection results of YOLOv11 model.

In [ ]:
print("VISUAL RESULTS ANALYSIS")


# Setup visualization components
color_palette = sv.ColorPalette.from_hex([
    "#FF6B6B", "#4ECDC4", "#45B7D1", "#96CEB4", "#FECA57", "#FF9FF3"
])

# Select interesting images for visualization
num_samples = min(9, len(all_predictions))
sample_indices = np.linspace(0, len(all_predictions)-1, num_samples, dtype=int)

result_images = []
titles = []

for idx in sample_indices:
    img_path = image_paths[idx]
    pred = all_predictions[idx]
    gt = all_ground_truth[idx]

    # Load image
    image = Image.open(img_path)

    # Calculate text scale based on image size
    text_scale = sv.calculate_optimal_text_scale(resolution_wh=image.size)
    thickness = sv.calculate_optimal_line_thickness(resolution_wh=image.size)

    # Create prediction labels
    pred_labels = [
        f"vehicle {confidence:.2f}"
        for confidence in pred.confidence
    ] if len(pred) > 0 else []

    # Annotate image with predictions
    annotated_image = image.copy()

    # Draw predictions in color
    if len(pred) > 0:
        bbox_annotator = sv.BoxAnnotator(color=color_palette, thickness=thickness)
        label_annotator = sv.LabelAnnotator(
            color=color_palette,
            text_color=sv.Color.WHITE,
            text_scale=text_scale
        )
        annotated_image = bbox_annotator.annotate(annotated_image, pred)
        annotated_image = label_annotator.annotate(annotated_image, pred, pred_labels)

    # Add ground truth boxes in white outline for comparison
    if len(gt) > 0:
        import cv2
        annotated_array = np.array(annotated_image)
        for box in gt.xyxy:
            x1, y1, x2, y2 = box.astype(int)
            cv2.rectangle(annotated_array, (x1, y1), (x2, y2), (255, 255, 255), max(1, thickness//2))
        annotated_image = Image.fromarray(annotated_array)

    result_images.append(annotated_image)

    # Create title with detection count
    gt_count = len(gt) if gt is not None else 0
    pred_count = len(pred) if pred is not None else 0
    titles.append(f"GT: {gt_count} | Pred: {pred_count}")

# Display results in grid
sv.plot_images_grid(
    images=result_images,
    grid_size=(3, 3),
    titles=titles,
    size=(15, 15)
)

print("\n Legend:")
print("  - Colored boxes: YOLOv11 predictions with confidence scores")
print("  - White outlines: Ground truth annotations")
print("  - GT: Ground truth count | Pred: Prediction count")

## 9. Results Summary and Export

Generate a summary of all metrics for research comparison.

In [ ]:
import json
from datetime import datetime

print("GENERATING RESULTS SUMMARY")

# Create comprehensive results dictionary
results_dict = {
    'model_info': {
        'architecture': 'YOLOv11',
        'approach': 'CNN-based',
        'classes': CLASS_NAMES,
        'num_classes': len(CLASS_NAMES),
        'detection_type': 'single-class'
    },
    'dataset_info': {
        'num_test_images': len(image_files),
        'image_resolution': '640x640',
        'processed_successfully': len(all_predictions)
    },
    'performance_metrics': {
        'mAP': {
            'mAP50': float(map_result.map50),
            'mAP50_95': float(map_result.map50_95),
            'mAP75': float(map_result.map75)
        },
        'binary_classification': {
            'precision': float(precision),
            'recall': float(recall),
            'f1_score': float(f1),
            'accuracy': float(accuracy)
        },
        'detection_counts': {
            'true_positives': int(true_positives),
            'false_positives': int(false_positives),
            'false_negatives': int(false_negatives),
            'true_negatives': int(true_negatives)
        }
    },
    'computational_efficiency': {
        'avg_inference_time': float(avg_time),
        'std_inference_time': float(std_time),
        'min_inference_time': float(min_time),
        'max_inference_time': float(max_time),
        'avg_fps': float(fps)
    },
    'evaluation_timestamp': datetime.now().isoformat()
}

# Display formatted summary
print("YOLOv11 EVALUATION SUMMARY")
print(f"\n Dataset: {len(image_files)} test images")
print(f" Target class: {CLASS_NAMES[0]}")
print("\nKEY PERFORMANCE METRICS:")
print(f"  - mAP@0.5: {map_result.map50:.3f}")
print(f"  - mAP@0.5:0.95: {map_result.map50_95:.3f}")
print(f"  - Precision: {precision:.3f}")
print(f"  - Recall: {recall:.3f}")
print(f"  - F1-Score: {f1:.3f}")
print(f"  - Accuracy: {accuracy:.3f}")
print("\n COMPUTATIONAL EFFICIENCY:")
print(f"  - Average inference time: {avg_time:.4f} seconds")
print(f"  - Average FPS: {fps:.2f}")
print(f"  - Processing speed: {len(image_files)/sum(inference_times):.2f} images/second")

print("\n DETECTION PERFORMANCE:")
print(f"  - True Positives: {true_positives}")
print(f"  - False Positives: {false_positives}")
print(f"  - False Negatives: {false_negatives}")
print(f"  - True Negatives: {true_negatives}")

# Save results to JSON file
with open('yolov11_single_class_results.json', 'w') as f:
    json.dump(results_dict, f, indent=2)

# Download results file
try:
    files.download('yolov11_single_class_results.json')

except:
    print("Results file saved locally (download from file panel)")

## 10. Prepare Data for Research Comparison

Generate data in format suitable for McNemar's test and statistical comparison with RF-DETR.

In [ ]:
print("PREPARING DATA FOR STATISTICAL COMPARISON")

def calculate_iou(box1, box2):
    """Calculate Intersection over Union of two bounding boxes"""
    x1_min, y1_min, x1_max, y1_max = box1
    x2_min, y2_min, x2_max, y2_max = box2

    # Calculate intersection
    inter_x_min = max(x1_min, x2_min)
    inter_y_min = max(y1_min, y2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_max = min(y1_max, y2_max)

    if inter_x_max <= inter_x_min or inter_y_max <= inter_y_min:
        return 0.0

    inter_area = (inter_x_max - inter_x_min) * (inter_y_max - inter_y_min)

    # Calculate union
    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max - x2_min) * (y2_max - y2_min)
    union_area = box1_area + box2_area - inter_area

    return inter_area / union_area if union_area > 0 else 0.0

# Create per-image detection results for McNemar's test
detection_results = []

for i, (pred, gt, img_path) in enumerate(zip(all_predictions, all_ground_truth, image_paths)):
    img_name = Path(img_path).stem

    # Calculate if detection was successful (IoU-based)
    has_ground_truth = len(gt) > 0
    has_prediction = len(pred) > 0

    # Check if any prediction matches ground truth (IoU > 0.5)
    detection_success = False
    if has_ground_truth and has_prediction:
        for gt_box in gt.xyxy:
            for pred_box, conf in zip(pred.xyxy, pred.confidence):
                if conf > 0.5:
                    iou = calculate_iou(gt_box, pred_box)
                    if iou > 0.5:
                        detection_success = True
                        break
            if detection_success:
                break
    elif not has_ground_truth and not has_prediction:
        detection_success = True  # Correct negative

    detection_results.append({
        'image_name': img_name,
        'has_ground_truth': has_ground_truth,
        'has_prediction': has_prediction,
        'detection_success': detection_success,
        'num_gt_objects': len(gt),
        'num_pred_objects': len(pred),
        'inference_time': inference_times[i]
    })

# Convert to DataFrame for easy analysis
results_df = pd.DataFrame(detection_results)

# Calculate success rate
success_rate = results_df['detection_success'].mean()
print(f"🎯 Overall Detection Success Rate: {success_rate:.3f}")

# Save detection results for McNemar's test
results_df.to_csv('yolov11_single_class_detection_results.csv', index=False)

print(f"\n Detection Results Summary:")
print(f"  - Images with ground truth: {results_df['has_ground_truth'].sum()}")
print(f"  - Images with predictions: {results_df['has_prediction'].sum()}")
print(f"  - Successful detections: {results_df['detection_success'].sum()}")
print(f"  - Success rate: {success_rate:.1%}")

# Generate research comparison summary
comparison_summary = f"""
YOLOv11 vs RF-DETR SINGLE-CLASS COMPARISON SUMMARY
==================================================

Research Project: CNN vs Vision Transformer for Satellite Vehicle Detection
Student: Abdullah Waraich (ID: 2401554)
Supervisor: Dr. Adrian Clark

ARCHITECTURE COMPARISON:
• YOLOv11: CNN-based single-stage detector
• RF-DETR: Vision Transformer-based detector

MODEL CONFIGURATION:
• Single class: {CLASS_NAMES[0]}
• Simplified evaluation pipeline
• Clean comparison without class imbalance issues

KEY FINDINGS:
• YOLOv11 mAP@0.5: {map_result.map50:.3f}
• RF-DETR mAP@0.5: [To be filled from RF-DETR evaluation]
• YOLOv11 inference time: {avg_time:.4f}s
• RF-DETR inference time: [To be filled from RF-DETR evaluation]
• YOLOv11 accuracy: {accuracy:.3f}
• RF-DETR accuracy: [To be filled from RF-DETR evaluation]

NEXT STEPS:
1. Run McNemar's Test for statistical significance
2. Compare computational efficiency
3. Analyze failure cases and strengths
4. Complete dissertation analysis

Files generated:
• yolov11_single_class_results.json
• yolov11_single_class_detection_results.csv
"""

# Save comparison summary
with open('yolov11_research_comparison.txt', 'w') as f:
    f.write(comparison_summary)

print(comparison_summary)

## 11. McNemar's Test Preparation

Prepare the data structure needed for McNemar's statistical test when comparing with RF-DETR.

In [ ]:
!pip install --upgrade scipy
from statsmodels.stats.contingency_tables import mcnemar
print("McNEMAR'S TEST PREPARATION")


# Create binary success/failure array for this model
yolo_results = results_df['detection_success'].values.astype(int)

print(f"\n YOLOv11 Detection Results Summary:")
print(f"  - Total images: {len(yolo_results)}")
print(f"  - Successful detections: {np.sum(yolo_results)}")
print(f"  - Failed detections: {len(yolo_results) - np.sum(yolo_results)}")
print(f"  - Success rate: {np.mean(yolo_results):.3f}")

# Save binary results for McNemar's test
mcnemar_data = {
    'image_names': results_df['image_name'].tolist(),
    'yolov11_success': yolo_results.tolist(),
    'yolov11_metrics': {
        'mAP50': float(map_result.map50),
        'success_rate': float(np.mean(yolo_results)),
        'precision': float(precision),
        'recall': float(recall),
        'f1_score': float(f1),
        'accuracy': float(accuracy)
    },
    'model_info': {
        'architecture': 'YOLOv11',
        'type': 'CNN-based',
        'classes': CLASS_NAMES
    }
}

with open('mcnemar_yolov11_single_class.json', 'w') as f:
    json.dump(mcnemar_data, f, indent=2)

print("\n McNemar's test data saved to: mcnemar_yolov11_single_class.json")

